# Background

Before data cleaning, we have performed data exploration and investigated the following topics:


*   the types of the features, i.e. binary, numerical, and categorical
*   the distribution of the features
*   the frequency of NaN values

The topics we have investigated above are criterias that help us decide if the target feature is relevant. The relevance of the features is stored in the excel file "Data_Description.xlsx".

# Import data

Since Colab section cannot keep files for long time, we used Google Drive to Store raw and cleaned dataset. Here, we connect to Google drive and mount to Colab directory.  
This step is optional. If you run this notebook locally, please put training dataset under ./data directory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Import library

In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

Declare data types to save memory

In [ ]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

Declare function to convert types for features.

In [ ]:
def convert_types(df):
    # Convert data types to reduce memory
    for c in df:
        col_type = str(df[c].dtypes)
        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

        # Convert objects to category
        if col_type == 'object':
            df[c] = df[c].astype('category')

        # numerics
        elif col_type in numerics:
            c_min = df[c].min()
            c_max = df[c].max()
            if col_type[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[c] = df[c].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[c] = df[c].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[c] = df[c].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[c] = df[c].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[c] = df[c].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[c] = df[c].astype(np.float32)
                else:
                    df[c] = df[c].astype(np.float64)

    return df

Load Data_Description.xlsx to identify relevant features.

In [ ]:
# target column
target = 'HasDetections'
# id from data set
data_id = 'MachineIdentifier'

training_set_file_path = './data/train.csv'
training_set_truncated_path = "./data/selected_training.csv"
EDA_file_path = "./data/EDA.xlsx"

In [ ]:
excel_table = pd.read_excel(EDA_file_path)
excel_table = excel_table[excel_table['Relevant']==1]
relevant_features = [el.replace('\xa0','') for el in excel_table['Feature']]

Define a function to randomly select 25% data from the original dataset to save memory and loading time.

In [ ]:
def process_and_save_by_chunk(file_path, output_file, chunk_size=10000, fraction=0.25):
    selected_rows = pd.DataFrame()

    for chunk in pd.read_csv(file_path, chunksize=chunk_size, low_memory=False):
        # Randomly sample a fraction of rows from each chunk
        sampled_chunk = chunk.sample(frac=fraction)
        selected_rows = pd.concat([selected_rows, sampled_chunk])

    # Save the selected rows to a new file
    selected_rows.to_csv(output_file, index=False)

Process the original dataset and save selected sample to current directory. This process will take around 10 mins.

In [ ]:
process_and_save_by_chunk(training_set_file_path, training_set_truncated_path)

In [ ]:
# use dask to load faster
ddf = dd.read_csv(training_set_truncated_path, dtype = dtypes)
df = ddf.compute()

In [ ]:
df.shape

(2230371, 83)

Eliminate irrelevant feature.

In [ ]:
df = df[relevant_features]

In [ ]:
df.shape

(2230371, 66)

In [ ]:
df = convert_types(df)

# Clean data in general

spliting up the features based on categorical, binary and numerical, safed prior into excel

In [ ]:
numerical_features = excel_table[excel_table['FeatureType']=='Numeric']['Feature'].reset_index(drop=True)
categorical_features = excel_table[excel_table['FeatureType']=='Category']['Feature'].reset_index(drop=True)
binary_features = excel_table[excel_table['FeatureType']=='Boolean']['Feature'].reset_index(drop=True)

In [ ]:
binary_features

0                                IsProtected
1                                      SMode
2                                   Firewall
3                 Census_HasOpticalDiskDrive
4                 Census_IsSecureBootEnabled
5                      Census_IsTouchEnabled
6                        Census_IsPenCapable
7    Census_IsAlwaysOnAlwaysConnectedCapable
8                               Wdft_IsGamer
9                              HasDetections
Name: Feature, dtype: object

For numeric data, we replace NaN values with "-1"

In [ ]:
for feature in numerical_features:
    df[feature] = df[feature].fillna(-1)

For binary feature, we reassign all NaN-Values to the most fequent feature

In [ ]:
for feature in binary_features:
    counts = df[feature].value_counts(dropna=True)
    top_value = counts.idxmax()
    df[feature] = df[feature].fillna(top_value)

In [ ]:
df['Wdft_IsGamer'].mode()

0    0.0
Name: Wdft_IsGamer, dtype: float16

For categorical features, we rename NaN-Values as '-1' in all features with tpye 'not category'.

In [ ]:
correct_feature_by_hand = []
for feature in categorical_features:
    t = pd.api.types.is_categorical_dtype(df[feature])
    if not t:
        # rename NaN-values to '-1'
        df[feature] = df[feature].fillna(-1)
    else:
        # add to list, to look at features
        correct_feature_by_hand.append(feature)

In [ ]:
correct_feature_by_hand

['MachineIdentifier',
 'EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'Platform',
 'Processor',
 'OsVer',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'SmartScreen',
 'Census_MDC2FormFactor',
 'Census_PrimaryDiskTypeName',
 'Census_ChassisTypeName',
 'Census_PowerPlatformRoleName',
 'Census_OSVersion',
 'Census_OSArchitecture',
 'Census_OSBranch',
 'Census_OSEdition',
 'Census_OSSkuName',
 'Census_OSInstallTypeName',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_GenuineStateName',
 'Census_ActivationChannel',
 'Census_FlightRing']

# Clean data by hand

Switch all feature name to lowercase.

In [ ]:
for feature in correct_feature_by_hand:
    df[feature] = df[feature].str.lower()

AvSigVersion

In [ ]:
df['AvSigVersion'].replace(['1.2&#x17;3.1144.0'], '1.2173.1144.0', inplace=True)

Census_ActivationChannel

In [ ]:
def rename_Census_ActivationChannel(x):
    x = x.lower()
    if 'oem' in x:
        return 'oem'
    elif 'volume' in x:
        return 'volume'
    elif 'retail' in x:
        return 'retail'
    else:
        return x

df['Census_ActivationChannel'] = df['Census_ActivationChannel'].astype(str)
df['Census_ActivationChannel'] = df['Census_ActivationChannel'].apply(rename_Census_ActivationChannel)
df['Census_ActivationChannel'] = df['Census_ActivationChannel'].astype('category')

Census_ChassisTypeName

In [ ]:
def rename_Census_ChassisTypeName(x):
    x = x.lower()
    if 'laptop' in x:
        return 'Notebook'
    elif 'other' in x:
        return 'unknown'
    else:
        return x

df['Census_ChassisTypeName'] = df['Census_ChassisTypeName'].fillna('unknown')
df['Census_ChassisTypeName'] = df['Census_ChassisTypeName'].astype(str)
df['Census_ChassisTypeName'] = df['Census_ChassisTypeName'].apply(rename_Census_ChassisTypeName)
df['Census_ChassisTypeName'] = df['Census_ChassisTypeName'].astype('category')
df['Census_ChassisTypeName'] = df['Census_ChassisTypeName'].cat.remove_unused_categories()

Census_FlightRing

In [ ]:
df['Census_FlightRing'] = df['Census_FlightRing'].astype('category')
df['Census_FlightRing'].replace(['disabled'], 'not_set', inplace=True)
df['Census_FlightRing'].replace(['osg', 'canary', 'invalid'], 'unknown', inplace=True)
df['Census_FlightRing'] = df['Census_FlightRing'].fillna('unknown')
df['Census_FlightRing'] = df['Census_FlightRing'].cat.remove_unused_categories()

Census_GenuineStateName

In [ ]:
df['Census_GenuineStateName'] = df['Census_GenuineStateName'].astype('category')
df['Census_GenuineStateName'].replace(['tampered'], 'unknown', inplace=True)
df['Census_GenuineStateName'] = df['Census_GenuineStateName'].fillna('unknown')
df['Census_GenuineStateName'] = df['Census_GenuineStateName'].cat.remove_unused_categories()

Census_MDC2FormFactor_new

In [ ]:
df['Census_MDC2FormFactor_new'] = df['Census_MDC2FormFactor']
correct_feature_by_hand.append('Census_MDC2FormFactor_new')

In [ ]:
def rename_Census_MDC2FormFactor_new(x):
    x = x.lower()
    if 'server' in x:
        return 'server'
    elif 'tablet' in x:
        return 'tablet'
    else:
        return x

df['Census_MDC2FormFactor_new'] = df['Census_MDC2FormFactor_new'].astype(str)
df['Census_MDC2FormFactor_new'] = df['Census_MDC2FormFactor_new'].apply(rename_Census_MDC2FormFactor_new)
df['Census_MDC2FormFactor_new'] = df['Census_MDC2FormFactor_new'].astype('category')
df['Census_MDC2FormFactor_new'] = df['Census_MDC2FormFactor_new'].cat.remove_unused_categories()

Census_OSEdition

In [ ]:
def rename_Census_OSEdition(x):
    x = x.lower()
    if 'core' in x:
        return 'core'
    elif 'pro' in x:
        return 'pro'
    elif 'enterprise' in x:
        return 'enterprise'
    elif 'server' in x:
        return 'server'
    elif 'home' in x:
        return 'home'
    elif 'education' in x:
        return 'education'
    elif 'cloud' in x:
        return 'cloud'
    else:
        return x

df['Census_OSEdition'] = df['Census_OSEdition'].astype('category')
df['Census_OSEdition'] = df['Census_OSEdition'].cat.add_categories(['unknown'])
df['Census_OSEdition'] = df['Census_OSEdition'].fillna('unknown')
df['Census_OSEdition'] = df['Census_OSEdition'].astype(str)
df['Census_OSEdition'] = df['Census_OSEdition'].apply(rename_Census_OSEdition)
df['Census_OSEdition'] = df['Census_OSEdition'].astype('category')
df['Census_OSEdition'] = df['Census_OSEdition'].cat.remove_unused_categories()

Census_OSSkuName

In [ ]:
def rename_Census_OSSkuName(x):
    x = x.lower()
    if 'core' in x:
        return 'core'
    elif 'pro' in x:
        return 'pro'
    elif 'enterprise' in x:
        return 'enterprise'
    elif 'server' in x:
        return 'server'
    elif 'home' in x:
        return 'home'
    elif 'education' in x:
        return 'education'
    elif 'cloud' in x:
        return 'cloud'
    else:
        return x

df['Census_OSSkuName'] = df['Census_OSSkuName'].astype(str)
df['Census_OSSkuName'] = df['Census_OSSkuName'].apply(rename_Census_OSSkuName)
df['Census_OSSkuName'] = df['Census_OSSkuName'].astype('category')
df['Census_OSSkuName'] = df['Census_OSSkuName'].cat.remove_unused_categories()

Census_PowerPlatformRoleName

In [ ]:
df['Census_PowerPlatformRoleName'] = df['Census_PowerPlatformRoleName'].astype('category')
df['Census_PowerPlatformRoleName'].replace(['unspecified'], 'unknown', inplace=True)
df['Census_PowerPlatformRoleName'] = df['Census_PowerPlatformRoleName'].fillna('unknown')
df['Census_PowerPlatformRoleName'] = df['Census_PowerPlatformRoleName'].cat.remove_unused_categories()

Census_PrimaryDiskTypeName

In [ ]:
df['Census_PrimaryDiskTypeName'] = df['Census_PrimaryDiskTypeName'].astype('category')
df['Census_PrimaryDiskTypeName'].replace(['unspecified'], 'unknown', inplace=True)
df['Census_PrimaryDiskTypeName'] = df['Census_PrimaryDiskTypeName'].fillna('unknown')
df['Census_PrimaryDiskTypeName'] = df['Census_PrimaryDiskTypeName'].cat.remove_unused_categories()

OsBuildLab

In [ ]:
df['OsBuildLab'] = df['OsBuildLab'].astype('category')
df['OsBuildLab'] = df['OsBuildLab'].cat.add_categories(['unknown'])
df['OsBuildLab'] = df['OsBuildLab'].fillna('unknown')

SmartScreen

In [ ]:
def rename_SmartScreen(x):
    x = x.lower()
    if 'promt' in x:
        return 'prompt'
    elif 'requireadmin' in x:
        return 'requireadmin'
    elif 'existsnotset' in x:
        return 'existsnotset'
    elif 'off' in x:
        return 'off'
    elif 'warn' in x:
        return 'warn'
    elif 'prompt' in x:
        return 'prompt'
    elif 'block' in x:
        return 'block'
    elif 'on' in x:
        return 'on'
    else:
        return 'unknown'

df['SmartScreen'] = df['SmartScreen'].astype('category')
df['SmartScreen'] = df['SmartScreen'].cat.add_categories(['unknown'])
df['SmartScreen'] = df['SmartScreen'].fillna('unknown')
df['SmartScreen'] = df['SmartScreen'].astype(str)
df['SmartScreen'] = df['SmartScreen'].apply(rename_SmartScreen)
df['SmartScreen'] = df['SmartScreen'].astype('category')
df['SmartScreen'] = df['SmartScreen'].cat.remove_unused_categories()

In [ ]:
df.shape

(2230371, 67)

# Export cleaned train data

In [ ]:
df.to_csv('./drive/MyDrive/train_clean.csv', index = False)